In [1]:
# this script reads and plots wrf output related to the paper describing the effect of upstream topography 
# on sundowner temporal variability

# Gert-Jan Duine, ERI UCSB, July/August 2020
# gertjan.duine@gmail.com / duine@eri.ucsb.edu


import pandas as pd # pandas is more labeled tabular oriented. works well if headers are provided in files.
import numpy as np
import netCDF4
from netCDF4 import Dataset
import wrf
import xarray as xr

# plotting features/packages
from cartopy import crs
from cartopy.feature import NaturalEarthFeature, COLORS
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature

from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)


In [2]:
# some functions
def calc_wspd_ms_to_knts(ms):
    knts = ms*1.9438445
    return knts


def calc_wspd(u, v):
    """Computes wind speed from u and v components"""        
    wspd = np.sqrt(u**2 + v**2) 
    return wspd


# this is for standard plots
def draw_basemap():

    # Create figure
    fig = plt.figure(figsize=(10,7)) 

    # Add plot axes and draw basemap
    ax = fig.add_subplot(111, projection=mapcrs)
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=mapcrs)

    # xticks
    ax.set_xticks(dx, crs=mapcrs)      
    lon_formatter = LongitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    # yticks
    ax.set_yticks(dy, crs=mapcrs)
    lat_formatter = LatitudeFormatter()
    ax.yaxis.set_major_formatter(lat_formatter)
    # tick params
    ax.tick_params(direction='out', labelsize=8.5, length=5, pad=2, color='black')    

    # Map features
    ax.add_feature(cfeature.LAND, facecolor='0.9') 
    ax.add_feature(cfeature.BORDERS, edgecolor='0.1', linewidth=0.7)
    ax.add_feature(cfeature.COASTLINE, edgecolor='k', linewidth=1.0)
    ax.add_feature(cfeature.STATES, edgecolor='0.1', linewidth=0.7)
    
    return fig, ax


In [4]:
pathHGT = '/home/voyager-sbarc/wrf/wrf401/sundowners/20170311/run-64452560-z0-'
folderListHGT=['1way', 'SRM075', 'SRM010']


path = '/home/voyager-sbarc/wrf/wrf401/sensSRM/output/201703/'
folderList=['SRM100', 'SRM075', 'SRM010']
runNames1=['(a) control','(b) 25% red.', '(c) 90% red.']
runNames2=['(d) control','(e) 25% red.', '(f) 90% red.']

fNameOut=['a','b','c','d','e','f']

# some plotting definitions, outside any loop
lonmin=-120.3
lonmax=-119.3
latmin=34.3
latmax=34.7
terLvls=np.arange(200,2200,200)
# tempLevs=np.arange(6,36,3) # a range b/w 0 and 33 deg C???
tempLevs=np.arange(0,33,3) # a range for temperature

# Tickmark Locations
dxInt = np.arange(lonmin,lonmax,0.2)
dx = dxInt[1:]
dy = np.arange(latmin,latmax,0.1)

c=1      # plot counter  : 1 for 14 PST, 4 for 20 PST
tees=[22] # times in file :22 for 14 PST, 4 for 20 PST
# 2017-03-11_00:00:00 for 14 PST, 2017-03-12_00:00:00 for 20 PST


for f in range(len(folderList)): # loop over foldernames????
    fName=path + str(folderList[f]) + '/wrfout_d04_2DVar_2017-03-11_00:00:00.nc'
    filepath = fName
    print('working on file ',filepath)
    wrf_file = netCDF4.Dataset(filepath)
#     ter = wrf.getvar(wrf_file, "ter", timeidx=-1)

    # plot with temperature in deg C at the surface and terrain height
    # Open the NetCDF file
    ncfile = Dataset(filepath)

    for t in tees: #np.arange(26,38,6): # time = XX
        if t==22:
            runNames=runNames1 # t 20 is 12 PST
        elif t==4:
            runNames=runNames2 # 
        
        print('working on plot',t, 'plot nr',str(c))
        # Get the variable
#         tc = getvar(ncfile, "tc",t)
        temp2m=wrf_file["T2"]
        temp2m=temp2m[t,:,:]-273.15
        timeWRF=getvar(ncfile,'Times',t)
        U10=wrf_file["U10"]
        U10=U10[t,:,:]
        V10=wrf_file["V10"]
        V10=V10[t,:,:]

        ### small computations with data
        wspd10=calc_wspd(U10,V10)
        U10_knts=calc_wspd_ms_to_knts(U10)
        V10_knts=calc_wspd_ms_to_knts(V10)

        # make string from the time in WRF using pandas
        ts = pd.to_datetime(to_np(timeWRF))
        tsPST = ts - pd.DateOffset(hours=8)
#         tWRFstr = ts.strftime('%Y%m%d_%H')
        tWRFstrPST=tsPST.strftime('%Y%m%d_%H')

        # Get the latitude and longitude points and cart projection. 
        # need to do like this, because somehow the 2D downloaded variables
        # do no work in this procedure
#         pathLala= '/home/voyager-sbarc/wrf/wrf401/sundowners/20170311/run-64452560-z0-'1way/wrfout_d04_2017-03-10_18:00:00'
        wrfHGT= netCDF4.Dataset(pathHGT+folderListHGT[f]+'/wrfout_d04_2017-03-10_18:00:00')
        uHGT,vHGT=getvar(wrfHGT,'uvmet10',0)
        cart_proj = get_cartopy(uHGT)
        # lons=getvar()
        lats, lons = latlon_coords(uHGT)
        ter=getvar(wrfHGT,'HGT',0)        
        # Get the cartopy mapping object
        cart_proj = get_cartopy(uHGT)

        del wrfHGT,uHGT,vHGT
        
        
        # Create a figure
        fig = plt.figure(figsize=(12,6))
        # Set the GeoAxes to the projection used by WRF
        ax = plt.axes(projection=cart_proj)
        ax.set_extent([lonmin,lonmax,latmin,latmax], crs=crs.PlateCarree())

        # xticks
        ax.set_xticks(dx, crs=crs.PlateCarree())
        lon_formatter = LongitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        # yticks
        ax.set_yticks(dy, crs=crs.PlateCarree())
        lat_formatter = LatitudeFormatter()
        ax.yaxis.set_major_formatter(lat_formatter)
        # tick params
        ax.tick_params(direction='out', labelsize=14, length=5, pad=2, color='black')    

        # Download and add the states and coastlines
        states = NaturalEarthFeature(category="cultural", scale="10m",
                                     facecolor="none",
                                     name="admin_1_states_provinces_shp")
        ax.add_feature(states, linewidth=.5, edgecolor="black")
        ax.coastlines('10m', linewidth=2)

        # Make the contour outlines and filled contours for the variable
        plt.contour(to_np(lons), to_np(lats), to_np(ter), terLvls, colors="black",
                    transform=crs.PlateCarree())
        plt.contourf(to_np(lons), to_np(lats), to_np(temp2m), tempLevs,
                     transform=crs.PlateCarree(),
                     cmap=get_cmap("jet"),alpha=0.6,extend='both')

        # Winds
#         ax.barbs(to_np(lons), to_np(lats), to_np(U10_knts), to_np(V10_knts), transform=crs.PlateCarree(), 
#                   color='k', linewidth=1, regrid_shape=16, pivot='middle',length=5.5)
        Q = ax.quiver(to_np(lons), to_np(lats), to_np(U10), to_np(V10), transform=crs.PlateCarree(), 
                  color='k', regrid_shape=16, pivot='middle')
        if c==1:
            qk = ax.quiverkey(Q, 0.8, 0.9, 10, '10 m s$^{-1}$', labelpos='E',
                   coordinates='figure',fontproperties=dict(size=16))

        # Add a color bar in plot 5.
        if c==4:
            cbar=plt.colorbar(ax=ax, shrink=.75) # shrink is the colorbar size
            cbar.set_label('Temperature (deg C)', fontsize=12)
      
        if (c==1 or c==2 or c==4 or c==5):
            ax.set_xticklabels([])
            
        if (c==4 or c==5 or c==6):
            ax.set_yticklabels([])
        else:
            plt.ylabel('Latitude $^\circ$N',fontsize=18)
        
        if (c==3 or c==6):
            plt.xlabel('Longitude $^\circ$W',fontsize=18)
        
        # Add the gridlines
#        ax.gridlines(color="grey", linestyle="dotted")

        if c==4:
            plt.title(runNames[f] + " " + tWRFstrPST + ":00 PST",fontsize=14)
        else:
            plt.title(runNames[f] + " " + tWRFstrPST + ":00 PST",fontsize=18)   
    
    
    #     plt.show()

        # save figure
#         fName='article_T2_winds_' + folderList[f] + '_' + tWRFstrPST + '_PST.pdf'
        fName='Fig05' + fNameOut[c-1] + '.pdf'
        fig.savefig(fName, bbox_inches='tight')#, dpi=100)
        fig.clf()
        plt.close() # close figure?
        
        c+=1
        del temp2m,U10_knts,V10_knts,timeWRF,U10,V10
    
    del ter, ax, lats,lons
print('done')

working on file  /home/voyager-sbarc/wrf/wrf401/sensSRM/output/201703/SRM100/wrfout_d04_2DVar_2017-03-11_00:00:00.nc
working on plot 22 plot nr 1
working on file  /home/voyager-sbarc/wrf/wrf401/sensSRM/output/201703/SRM075/wrfout_d04_2DVar_2017-03-11_00:00:00.nc
working on plot 22 plot nr 2
working on file  /home/voyager-sbarc/wrf/wrf401/sensSRM/output/201703/SRM010/wrfout_d04_2DVar_2017-03-11_00:00:00.nc
working on plot 22 plot nr 3
done
